<a href="https://colab.research.google.com/github/steghoja/Google2Csv/blob/master/Google2Csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GoogleScraper

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [0]:
def simpleGoogleSearch(query, start):
  results = []

  query = query.replace(' ', '+')
  URL = f"https://google.com/search?q={query}&start={start}"

  # desktop user-agent
  USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
  
  headers = {"user-agent" : USER_AGENT}
  resp = requests.get(URL, headers=headers)

  if resp.status_code == 200:
    soup = BeautifulSoup(resp.content, "html.parser")

    for g in soup.find_all('div', class_='rc'):
      anchors = g.find_all('a')

      if anchors:
        link  = anchors[0]['href']
        title = g.find('h3').text
        description = g.find('span', class_='aCOpRe')
        date = description.find('span', class_='f')
        if date:
          date = date.text[:-2]
        item  = {"title": title, "date": date, "description": description.text, "link": link}
        results.append(item)

  return results

In [0]:
def googleToPandas(googleQuery):
  resultsCounter  = 0
  resultsList     = []

  while True:
    pageResults = simpleGoogleSearch(googleQuery, resultsCounter)
    
    if not pageResults: break
    else: 
      resultsList.extend(pageResults)
      resultsCounter = resultsCounter + 10

  return pd.DataFrame(resultsList)

In [0]:
googleSearchQuery = "" #@param {type:"string"}

In [0]:
results = googleToPandas(googleSearchQuery)
results

In [0]:
results.to_csv('GoogleResults.csv', index=False)
results.to_excel('GoogleResults.xlsx', index=False)
results.to_json('GoogleResults.jsonl', orient='records', lines=True)